In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from jaxtyping import Float
from chbmit.chbmit import FilteredCMP, STFT

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
chb01 = FilteredCMP(600, 1, '', sop=3600, regression=True)

TypeError: FilteredCMP.__init__() missing 1 required positional argument: 'sph'

In [28]:
class MultiHead(nn.Module):
  def __init__(self, ):
    super().__init__()
    self.shared_layers = nn.Sequential(nn.Conv2d(3, 96, (11, 11), 4),
                                       nn.MaxPool2d(2),
                                       nn.Conv2d(96, 384, (5, 5), padding='same'), 
                                       nn.MaxPool2d(2),
                                       nn.Conv2d(384, 384, (3, 3), padding='same'),
                                       nn.Conv2d(384, 384, (3, 3), padding='same'),
                                       nn.Conv2d(384, 256, (3, 3), padding='same'),
                                       nn.MaxPool2d(2),
                                       nn.Flatten(),
                                       nn.Linear(9216, 4096),
                                       nn.ReLU(),
                                       nn.Linear(4096, 4096),
    )
    self.classification_head = nn.Sequential(
      nn.Linear(4096, 1000),
      nn.ReLU(),
      nn.Linear(1000, 1000),
      nn.ReLU(),
      nn.Linear(1000, 2)
    )

    self.regression_head = nn.Sequential(
      nn.Linear(4096, 1000),
      nn.ReLU(),
      nn.Linear(1000, 1000),
      nn.ReLU(),
      nn.Linear(1000, 1)
    )

    self.variance_head = nn.Sequential(
      nn.Linear(4096, 1000),
      nn.ReLU(),
      nn.Linear(1000, 1000),
      nn.ReLU(),
      nn.Linear(1000, 1)
    )

  def forward(self, data): 
    shared_output = self.shared_layers(data)
    return (self.classification_head(shared_output), self.regression_head(shared_output), self.variance_head(shared_output))
